# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación
### Minería de datos: Curso 2020-2021
* José Gabriel Ruiz Gomez
* Francisco Javier Vicente Martínez

# 1. Preliminares

Cargamos las librerias necesarias

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from mlxtend.feature_selection import ColumnSelector

import numpy as np
import seaborn as sns
sns.set()
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

Fijamos la semilla para que el experimento sea reproducible:

In [ ]:
seed = 27912

# Base de datos Pima

# 2. Acceso y almacenamiento de datos

La base de datos Pima Indians Diabetes contiene 768 entradas que corresponden a mujeres de descendencia india de al menos 21 años de edad. La base de datos las clasifica segun tengan diabetes o no con la variable de clase:

* `Outcome`

Ésta puede tomar el valor 0 o 1. Las variables predictoras para este problema son:

* `Pregnancies`: Representa el numero de veces que quedo embarazada
* `Glucose`: Concentracion de glucose en plasma a 2 horas de un examen de tolerancia de glucosa
* `BloodPresure`: Presion arterial diastólica (mm Hg)
* `SkinThickness`: Grosor de un pliegue de piel del triceps (mm)
* `Insulin`: Serum de insulina de 2 horas (mu U/ml)
* `BMI`: Indice de masa corporal
* `DiabetesPedigreeFunction`: Indice obtenido a partir de familiares que padecen diabetes
* `Age`: Edad 

Cargamos los datos de `Pima`: 

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index = False
target = "Outcome"

data = utils.load_data(filepath, index, target)

Se ha especificado la variable clase pero en esta base de datos no hay ninguna variable que sirva de identificador.

Comprobamos que se han cargado bien los datos, la funcion `head` puede que nos de una muestra sesgada pero mi objetivo es ver simplemente si el objeto data se ha creado correctamente:

In [ ]:
data.head(5)

Dividimos los datos en variables predictoras y resultado.

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

Comprobamos que se han divido correctamente.

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Para evitar el sobreajuste a los datos vamos a dividirlos en conjunto de entrenamiento y de prueba en un ratio de 70% entrenamiento y 30% de prueba:

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      shuffle=True,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

El conjunto de datos de pima no parece seguir ningun tipo de orden pero he decidido aleatorizarlo (`shuffle=True`) de todas formas.

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

Volvemos a juntar las variables predictoras con las objetivo para el analisis exploratorio:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)
data_test = utils.join_dataset(X_test, y_test)

In [ ]:
data_train.sample(5, random_state=seed)

# 3. Análisis exploratorio de los datos

### Descripcion del conjunto de datos

In [ ]:
data_train.shape

Nuestro conjunto de entrenamiento se compone de 537 casos con 9 variables, 8 predictoras 1 objetivo.

In [ ]:
data_train.info(memory_usage=False)

Todas las variables son numericas, excepto la variable objetivo.
Todas las variables son continuas, aunque el numero de embarazos podriamos considerarla discreta ya que por su naturaleza no va a tener muchos valores distintos.

In [ ]:
y_train.cat.categories

Nuestra variable clase tiene dos estados: 0 representa que no tiene diabetes y 1 representa que tiene diabetes.

### Visualizacion de las variables

Ahora que ya conocemos el conjunto de datos debemos analizar la distribucion de las variables. En este caso vamos a analizar las variables mediante metodos univariados, en este caso concreto histogramas, ya que todas nuestras variables predictoras son numericas y un diagrama de barras para nuestra variable objetivo.

In [ ]:
utils.plot_histogram(data_train)

Todos los atributos parecen distribuciones normales, en el caso de `pregnancies`, `insulin`, `diabetesPedigreeFunction` y `age` son distribuciones asimetricas.

Los atributos `pregnancies`, `glucose`, `bloodPresure`, `BMI`, `DiabetesPedigreeFunction` y `Age` contienen algunos outliers.

Variables como `glucose`, `bloodPresure` y `BMI` tienen unos cuantos valores perdidos, `Skin thickness` tiene muchos valores perdidos.

Los valores perdidos estan representados por un 0, el caso de la variable `Insulin` es un tanto problematico porque los pacientes con diabetes de tipo 1 no producen nada de insulina, esto hace complicado distinguir valores perdidos de valores de insulina 0 reales, con lo cual es posible que no podamos usar esta variable, pero para esta practica voy a tratarlos todos como valores perdidos.

In [ ]:
utils.plot_barplot(data_train)

La muestra no esta balanceada hay mas casos en los que `Outcome` es 0.

A continuacion vamos a hacer un analisis multivariado con una matriz de gráficos. Para ver relaciones entre variables.

In [ ]:
sp = utils.plot_pairplot(data_train, target="Outcome")
sp.update_layout(width=1400, height=1400, hovermode='closest')
sp.show()

A pesar de no poder verse bien hay un par de variables que tienen cierto poder discriminador combinadas con el resto: `Glucose` y `Pregnancies`. 

Como se puede ver, los valores para las dos posibilidades de la variable clase estan muy entremezcados, al no haber una distincion clara entre ambas podemos asumir que el modelo no va a salir demasiado bueno.

Otras variables como `DiabetesPedigreeFunction` o `Insulin` seguramente sea mejor utilizarlas de forma independiente.

Este grafico al tener tantas variables y estar todos los casos tan apelotonados es poco util, para poder cuantificar la relacion entre las variables voy a utilizar un heatmap de coorelacion:

In [ ]:
utils.px.imshow(data_train.corr())

Podemos ver que `Age` y `Pregnancies` tienen bastante correlacion, ademas tambien `BMI` esta coorelacionado con `skinThickness`, `Glucose` y `BloodPresure`, lo cual es logico.

Por otra parte tambien se puede observar una coorelacion positiva entre `skinThicness` e `Insulin` pero esto puede que se deba a la gran cantidad de valores perdidos (0) que tienen ambas.

# 4. Preprocesamiento de datos

Para poder obtener un modelo que tenga sentido de nuestros datos primero tenemos que abordar dos problemas: la discretizacion (dado el caso) y los valores perdidos.

### Valores perdidos

Tenemos el problema de que no podemos simplemente imputar los valores perdidos dentro del pipeline porque los valores perdidos se estan representando con un 0 y hay variables en las que 0 es un valor válido, sin ir mas lejos nuestra variable clase tiene 0 o 1, si sustituimos todos los 0 nos cargamoe el problema y todos los modelos van a estar mal, por lo cual no podemos utilizar por si solo un `SimpleImputer`.

En lugar de esto lo que vamos a utilizar es otro `estimator` que, en teoria, nos va a permitir aplicar nuestro `simpleImputer` tan solo a determinadas columnas, se trata de un `ColumnTransformer`.

In [ ]:
simpleImputer = SimpleImputer(missing_values=0, strategy='most_frequent')
cols = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

imputer = ColumnTransformer(
    [("ImpMissing", simpleImputer, cols)])

Hemos utilizado como estrategia para el `SimpleImputer` la moda porque en los datos hay outliers y la media se ve muy afectada por estos, la mediana se ve menos afectada, pero como hay valores tan extremos ceemos que es mejor utilizar la moda en su lugar. La moda, aunque menos, tambien se ve algo afectada por valores extremos.

Tambien podriamos haber usado correlacion con otras variables, pero no se ve una correlacion tan fuerte como para que valga la pena usar este método.

### Discretizacion

Vista la distribucion de outcome y la naturaleza de los datos tiene mas sentido discretizar con la estratiegia de las k-medias en 2 intervalos.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="kmeans")

# 5. Algoritmos de clasificación

Creamos el pipeline para todos estimadores, primero siempre poniendo el transformador que imputa los valores perdidos y luego, en su caso, el transformador para la discretización. 

### Algoritmo Zero-R

In [ ]:
zero_r_model = make_pipeline(imputer, DummyClassifier(strategy="most_frequent"))

### Algoritmo CART

In [ ]:
tree_model = make_pipeline(imputer, DecisionTreeClassifier(random_state=seed))

In [ ]:
discretize_tree_model = make_pipeline(imputer, discretizer, DecisionTreeClassifier(random_state=seed))

En este caso no he modificado los hiperparametros de los modelos porque el hiperparameter tunning no forma parte de esta practica, asi que estan por defecto

# 6. Evaluacion de modelos

In [ ]:
#Variables para analizar los modelos resultantes
Algoritmos=[]
Metricas=[]

### Zero-R

In [ ]:
y_pred=utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)
Algoritmos.append("ZeroR\t")
Metricas.append(utils.confMatMetricas(y_test,y_pred, 1, 0))

No hay mucho que decir del Zero-R, el porcentaje de acierto va a ser siempre exactamente igual a la proporcion de casos de la clase mayoritaria, se pueden utilizar los resultados del Zero-R como baseline, el modelo que se acierte mas o menos lo mismo o menos que un Zero-R no vale la pena.

### Arbol de clasificación

In [ ]:
y_pred=utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)
Algoritmos.append("ArbolClas")
Metricas.append(utils.confMatMetricas(y_test,y_pred, 1, 0))

El arbol de clasificacion sin discretizar no lo ha hecho mucho mejor que el Zero-R, pero por lo menos clasifica algunos "1" correctamente

### Arbol de clasificación discretizado

In [ ]:
y_pred=utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)
Algoritmos.append("ArbolClasDiscr")
Metricas.append(utils.confMatMetricas(y_test,y_pred, 1, 0))

Discretizando hemos conseguido mejorar la precision.

# Conclusiones

Respecto a los clasificadores obtenidos, son todos bastante malos, pero en base a los resultados discretizando hemos obtenido bastante mas precision aunque a costa de la tasa de verdaderos positivos, este clasificador podria valer para etiquetar correctamente verdaderos negativos, pero eso lo hace mejor el Zero-R.

De los 3 clasificadores el mejor para clasificar verdaderos positivos es el arbol sin discretizar, aunque tenga una precision parecida al Zero-R y bastantes falsos positivos, para este tipo de problema, desde el punto de vista de la salud creo que seria mejor etiquetar mal a los negativos que a los positivos.

In [ ]:
utils.f1Tabla(Algoritmos, Metricas)

Como se puede ver en la tabla anterior el arbol de clasificacion discretizado tiene mejor precision, pero, tal y como habiamos dicho antes, en nuestro caso el árbol de clasificacion sin discretizar es mejor para nuestro caso ya que tiene bastante mejor recall y en un problema como este en el que se estan prediciendo enfermedades preferimos clasificar bien antes a los positivos que a los negativos. Ademas la F1 score da mejor resultado para el arbol sin discretizar también.

# Base de datos Wisconsin

# 2. Acceso y almacenamiento de datos

El conjunto datos a utilizar es `Breast Cancer Wisconsin`. 
Contiene 546 muestras que se clasifican en dos tipos de tumores:
* `B = Benigno`
* `M = Maligno`

Para cada tumor se han realizado una serie de mediciones correspondientes a las variables predictoras del problema:
* `radius` : media de distancias entre el centro de los puntos al perímetro
* `texture` : desviación estandard de los valores de escala de grises
* `perimeter` : perímetro
* `area` : área
* `smoothness` : variación local en la longitud de los radios
* `compactness` : perímetro^2 / area - 1.0
* `concavity` : severidad de las porciones cóncavas del contorno
* `concave points` : numero de las porciones cóncavas del contorno
* `symetry` : simetría
* `fractal dimension` : aproximación de la línea de costa - 1

En las tablas se representa la media, la desviación típica y un valor "worst" que es la media de los tres mayores valores. Resultando 30 variables distribuidas de forma que el campo 3 es Media del Radio, el campo 13 desviación típica del Radio y campo 23 es "worst" del radio.

El objetivo sería clasificar una nueva instancia (cuya clasificación es desconocida) en función de sus variables.

Comenzamos cargando el conjunto de datos `Breast Cancer Wisconsin`:

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

data = utils.load_data(filepath, index, target)
#Eliminamos columna Unnamed
data = data.loc[:,data.columns !="Unnamed: 32"]
data

Dividimos el conjunto de datos en dos subconjuntos, uno con variables predictoras (X) y otro con la variable objetivo (y).

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

Además debemos separar nuestro conjunto de datos en dos:

* Una muestra de entrenamiento (típicamente, 70%)
* Una muestra de prueba (típicamente, 30%)

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                     stratify=y, 
                                                     random_state=seed,
                                                     train_size=train_size)

Para facilitar el análisis exploratorio de datos, volvemos a juntar las variables predictoras con la variable clase. Comenzamos con el conjunto de datos entrenamiento:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

Contiunamos con el conjunto de datos de prueba

In [ ]:
data_train

In [ ]:
data_test = utils.join_dataset(X_test, y_test)

# 3. Análisis exploratorio de los datos

Antes de comenzar el preprocesamiento es interesante observar las propiedades del conjunto de datos, analizando sus variables y la interacción entre estas. No obstante, no podemos usar el formato tabular directamente puesto que para un humano es casi imposible extraer conclusiones a partir del análisis de valores numéricos. Por ello, nos apoyaremos en gráficos y estadísticos.

### Descripción del conjunto de datos

Antes de realizar cualquier operación es fundamental conocer nuestro problema. Hay dos dimensiones básicas que deben ser exploradas:

* Número de casos
* Número de variables
    * Tipo de las variables: Continuas (t.c.c. numéricas) o discretas (t.c.c. categóricas)

Para ello, consultaremos las estructuras de datos correspondientes.

In [ ]:
print(X_train.shape)
print(y_train.shape)

Tal y como se puede observar, el conjunto de datos de entrenamiento está formado por 398 casos y 31 variables (30 variables predictoras y 1 variable clase)

Para conocer cuál es el tipo de las variables, recurrimos al método `info`:

In [ ]:
data_train.info(memory_usage=False)

Todas las variables son numéricas, excepto la variable objetivo. 

In [ ]:
y_train.cat.categories

La variable clase tiene dos estados *(M y B)*.

### Visualización de las variables

Una vez conocemos con más detalle el conjunto de datos de entrenamiento, lo que debemos hacer es representar y analizar las distribuciones de las variables. Para ello, utilizaremos métodos univariados, esto es, histogramas para las variables numéricas y diagramas de barras para las variables categóricas. En particular:

* Un histograma muestra la densidad de ejemplos para los distintos valores de una variable numérica.
* Un diagrama de barras representa la frecuencia de cada estado de una variable categórica.

In [ ]:
utils.plot_histogram(X_train)

In [ ]:
utils.plot_barplot(data)

Podemos ver que la clase objetivo no tiene el mismo número de variables para cada clasificación por lo tanto no es un problema que se encuentre balanceado. 

In [ ]:
data_train.describe(include="category")

In [ ]:
data_train.describe(include="number")

Para que las gráficas sean más representativas dividimos el conjunto de datos de entrenamiento en tres subconjuntos, que representarán cada uno los valores de las medias (means_data), los valores de el error standard (SE_data) y los valores "worst" (worst_data)

In [ ]:
means_data = data_train[["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean", "concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","diagnosis"]]
SE_data = data_train[["radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se", "concavity_se","concave points_se","symmetry_se","fractal_dimension_se","diagnosis"]]
worst_data = data_train[["radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst","compactness_worst", "concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst","diagnosis"]]

Ahora representaremos en diferentes gráficas los datos para poder obtener más conocimiento

In [ ]:
sns.heatmap(means_data.corr(), annot=True)

In [ ]:
sns.heatmap(SE_data.corr(), annot=True)

In [ ]:
sns.heatmap(worst_data.corr(), annot=True)

Muchos de los valores guardan alta correlación entre sí, por ejemplo radio con perímetro y área o puntos cóncavos con la concavidad, lo cual es lógico porque son atributos que van bastante de la mano semánticamente hablando.

In [ ]:
sp = utils.plot_pairplot(means_data, target="diagnosis")
sp.update_layout(width=1400, height=1400, hovermode='closest')
sp.show()

In [ ]:
sp = utils.plot_pairplot(SE_data, target="diagnosis")
sp.update_layout(width=1400, height=1400, hovermode='closest')
sp.show()

In [ ]:
sp = utils.plot_pairplot(SE_data, target="diagnosis")
sp.update_layout(width=1400, height=1400, hovermode='closest')
sp.show()

Atendiendo a las gráficas podemos fijarnos en que se forman en todos los atributos como dos grandes grupos de datos que discriminarían la clasificación B de la clasificación M por lo que podría ser interesante discretizar en 2 intervalosn estos datos. 

# 4. Preprocesamiento de datos

Dentro del preprocesamiento de datos, podemos destacar las siguientes tareas:

* Limpieza de datos (imputación de valores perdidos, suavizado del ruido, etc.)
* Integración de datos (a partir de múltiples fuentes)
* Transformación de datos (normalización, construcción, etc.)
* Reducción de datos (discretización de variables numéricas, selección de variables, selección de instancias, etc.)

### Valores perdidos

Para poder usar un pipeline y poder distinguir las variables con valores perdidos tenemos que "modificar" toda la base de datos, realmente no vamos a modificar los datos, simplemente vamos a indicar como nan los valores perdidos, de esta forma podemos aplicar el mismo pipeline al conjunto de entrenamiento, al conjunto de test y a los datos que vengan despues.

### Discretización

Como hemos visto, la discretización permite transformar variables numéricas en categóricas, siendo este paso beneficioso para algunos algoritmos de aprendizaje, pues permite que modelos lineales resuelvan problemas no lineales.

`scikit-learn` permite realizar tres tipos de discretización (`strategy`) mediante el transformador `KBinsDiscretizer`:

* `uniform`: Igual anchura.
* `quantile`: Igual frecuencia.
* `kmeans`: Discretización basada en k-medias.

Tras el análisis exploratorio de datos realizado previamente, parece lógico realizar una discretización en 2 intevalos de igual anchura:

In [ ]:
imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="kmeans")

Eliminar del conjunto de datos:

# 5. Algoritmos de clasificación

### Algoritmo Zero-R

In [ ]:
zero_r_model = make_pipeline(imputer, DummyClassifier(strategy="most_frequent"))

### Algoritmo CART 

In [ ]:
tree_model = make_pipeline(imputer, DecisionTreeClassifier(random_state=seed))

In [ ]:
discretize_tree_model = make_pipeline(imputer, discretizer, DecisionTreeClassifier(random_state=seed))

# 6. Evaluacion de modelos

### Zero-R

In [ ]:
Algoritmos.clear()
Metricas.clear()

In [ ]:
y_pred=utils.evaluate(zero_r_model,
              X_train, X_test,
              y_train, y_test)
Algoritmos.append("ZeroR\t")
Metricas.append(utils.confMatMetricas(y_test,y_pred, 'M', 'B'))

### Arbol de clasificación

In [ ]:
y_pred=utils.evaluate(tree_model,
              X_train, X_test,
              y_train, y_test)
Algoritmos.append("ArbolClas")
Metricas.append(utils.confMatMetricas(y_test, y_pred, 'M', 'B'))

In [ ]:
y_pred=utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)
Algoritmos.append("ArbolClasDiscr")
Metricas.append(utils.confMatMetricas(y_test,y_pred, 'M', 'B'))

# Conclusiones

Para esta base de datos, los clasificadores obtenidos tienen una mayor proporción de aciertos, no obstante, tenemos que fijarnos en la tasa de verdaderos positivos, ya que de nuevo es una clasificación basada en datos de salud donde nos interesa minimizar el error en éste parámetro.


In [ ]:
utils.f1Tabla(Algoritmos, Metricas)

De nuevo basándonos en el Recall podemos ver que el algoritmo que mejor nos clasifica los resultados sería el Arbol de Clasificación sin discretizar. 